# Security Onion DNS

## Import Python Modules

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Q, Search
import pandas as pd

### Suppress Certificate Warning for Self-Signed Cert

In [2]:
import urllib3
urllib3.disable_warnings(category=urllib3.exceptions.InsecureRequestWarning)

### Initialize Search Client

In [3]:
es = Elasticsearch(['https://172.20.2.10:9200'],
ca_certs=False,verify_certs=False, basic_auth=('jupyter','seconion'))
searchContext = Search(using=es, index='.ds-logs-*', doc_type='doc')

/opt/conda/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:400: SecurityWarning: Connecting to 'https://172.20.2.10:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


### Prepare Search Query

In [13]:
q = Q('terms', tags=['dns']) & Q('range', **{'@timestamp': {'gte': 'now-24h', 'lt': 'now'}})
#f = 
src = ('@timestamp', 'source.ip', 'source.port', 'destination.ip', 'destination.port', 'dns.query.name', 'dns.query.type_name', 'dns.response.code_name')
s = searchContext.query(q).source(src)#.filter(f)

### Query and build dataset for DNS events

In [14]:
response = s.execute()
if response.success():
 df = pd.json_normalize([hit.to_dict() for hit in s.scan()])
#  df = pd.DataFrame(([d['event']['dataset'], d['process']['executable'], d['file']['target']] for d in s))
#  df = pd.DataFrame(([d['event']['dataset'], d['@timestamp'], d['source']['ip']] for d in s))
#  df.columns=['Dataset','Timestamp','Source.IP']
#  df = pd.DataFrame([d.to_dict() for d in s.scan()])
df
#print(df)

,@timestamp,dns.query.type_name,dns.query.name,dns.response.code_name,destination.port,destination.ip,source.port,source.ip
0,2024-09-24T18:52:24.784Z,HTTPS,ssl.gstatic.com,NOERROR,53.0,1.0.0.1,53840.0,172.20.2.2
1,2024-09-24T18:51:46.938Z,AAAA,securityonion-managersearch.local,NaN,5353.0,ff02::fb,5353.0,fe80::53f0:2dc7:ed88:dc10
2,2024-09-24T18:52:35.382Z,A,otx.alienvault.com,NOERROR,53.0,8.8.8.8,48452.0,172.20.2.10
3,2024-09-24T18:52:35.382Z,AAAA,otx.alienvault.com,NOERROR,53.0,8.8.8.8,35007.0,172.20.2.10
4,2024-09-24T18:52:08.012Z,HTTPS,www.google.com,NaN,53.0,1.0.0.1,57810.0,172.20.2.2
...,...,...,...,...,...,...,...,...
51949,2024-09-23T20:49:36.719Z,A,v10.events.data.microsoft.com,NOERROR,53.0,172.20.3.1,61316.0,172.20.3.19
51950,2024-09-23T20:49:32.244Z,PTR,_microsoft_mcc._tcp.local,NaN,5353.0,ff02::fb,5353.0,fe80::53f0:2dc7:ed88:dc10
51951,2024-09-23T20:49:40.382Z,A,edge.skype.com,NOERROR,53.0,172.20.3.1,51536.0,172.20.3.19
51952,2024-09-23T20:49:33.235Z,PTR,_microsoft_mcc._tcp.local,NaN,5353.0,224.0.0.251,5353.0,172.20.2.98
